# perceptual decision-making


In [1]:
import numpy as np 
import matplotlib.pyplot as plt
import torch
import torch_directml
from torch.utils.data import DataLoader
from sklearn.mixture import GaussianMixture 
import dataset.generate_dataset as generate_dataset
import models.neural_networks as neural_networks 
import models.latent_dynamics as latent_dynamics
import evaluation.visualise_dynamics as visualise_dynamics
import evaluation.visualise_connectivity as visualise_connectivity
import matplotlib.colors as mcolors
import scipy 

device = torch_directml.device()
print('Using {}'.format(device))

np.random.seed(0)
torch.manual_seed(2)
list_colors = list(mcolors.TABLEAU_COLORS)

## 1. Generate data.

In [ ]:
batch_size = 32
num_trials = batch_size * 250
t_onset = 0.1 # 5 / sampling_rate # [s]
t_offset = 0.9 # 45 / sampling_rate # [s]
t_max = 1500/1000 # [s]
sampling_rate = 1/(20/1000) #[Hz]
num_trial_to_plot = 5 # should be less than batch_size
stimulus_strength_list  = \
    3.2/100 * 2 ** np.arange(0, 5)
stimulus_strength_list = \
    np.append(stimulus_strength_list, (-stimulus_strength_list))

PDM_dataset_train = generate_dataset.PDMStimulus(num_trials, t_onset, t_offset, t_max, stimulus_strength_list, sampling_rate)
PDM_dataset_test = generate_dataset.PDMStimulus(int(num_trials/100), t_onset, t_offset, t_max, stimulus_strength_list, sampling_rate)

fig = PDM_dataset_train.plot_stimulus(np.arange(num_trial_to_plot))
plt.show()

In [ ]:
train_dataloader = DataLoader(PDM_dataset_train, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(PDM_dataset_test, batch_size=batch_size, shuffle=True)

# check a batch 
for i_batch, sample_batched in enumerate(train_dataloader):
    print(i_batch, sample_batched['stimulus'].size(),
          sample_batched['label'].size())
    
    batch_size = len(sample_batched['stimulus'])
    
    # observe 4th batch and stop.
    if i_batch == 3:
        for i_trial in range(num_trial_to_plot):
            plt.plot(PDM_dataset_train.time, sample_batched['stimulus'][i_trial, :], \
                    label='trial {:d}, label {:d}'.format(int(i_trial), int(sample_batched['label'][i_trial])))
        plt.xlim(0, max(PDM_dataset_train.time))
        plt.xlabel('time [s]')
        plt.ylabel('u')
        plt.legend()
        plt.title('Batch {}'.format(i_batch))
        plt.show()
        break

## 2. Low rank RNN  

In [ ]:
input_size = 1
hidden_size = 128
output_size = 1
rank = 1
time_step = 20/1000 # s 
tau = 100/1000 # s
model = neural_networks.LowRankRNN(
    input_size, hidden_size, 
    output_size, rank, 
    time_step, tau).to(device)
print(model)

Run simulation & Visualise a time course of each hidden node.  

In [ ]:
sample_batched = next(iter(train_dataloader))
print('try simulation for the first batch')
print('shape of stimulus: ', sample_batched['stimulus'].size())
print('shape of label:', sample_batched['label'].size())

visualise_dynamics.plot_hidden_nodes_single_trial(
    device, model, t_max, PDM_dataset_train, train_dataloader)

## 3. Train network

In [ ]:
loss_fn = torch.nn.MSELoss(reduction='mean')
learning_rate = 5e-3
loss_min = 5e-2
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
time_step_for_loss = 15 #[-] not in sec    

In [ ]:
def train(dataloader, model, loss_fn, optimizer):
    num_epoches = len(dataloader)
    loss_history = []
    for i_batch, sample_batched in enumerate(dataloader):
        #print(i_batch, sample_batched['stimulus'].size(),
        #      sample_batched['label'].size())      
              
        _batch_size = len(sample_batched['stimulus'])
        hidden_state = model.init_hidden(_batch_size)        
        loss = 0 
        optimizer.zero_grad()
        #print(sample_batched['stimulus'].shape)
        num_sample_points = sample_batched['stimulus'].shape[1]
        
        for i_time in range(num_sample_points):
            #print(sample_batched['stimulus'].shape)
            input_data = sample_batched['stimulus'][:, i_time].unsqueeze(dim=1).to(device)
            label_data = sample_batched['label'].float().to(device)
            hidden_state = hidden_state.to(device)
            output, hidden_state = model(input_data, hidden_state)
            
            if i_time >=(num_sample_points - time_step_for_loss):
                #print(output.shape)
                #print(loss_fn(output.squeeze().to(device), label_data))
                loss += loss_fn(output.squeeze().to(device), label_data)
        
        # Backpropagation
        loss.backward()
        optimizer.step()

        loss, current = loss.item(), (i_batch+1) * _batch_size
        
        if i_batch % 100 == 0:
            print(f"loss: {loss:>7f}  [{current:>5d}/{num_epoches*_batch_size:>5d}]")
        
        loss_history.append(loss)

    return loss_history 

def test(dataloader, model, loss_fn):
    num_epoches = len(dataloader)
    model.eval()
    test_loss = 0 
    correct = 0
    with torch.no_grad():
        for i_batch, sample_batched in enumerate(dataloader):
            _batch_size = len(sample_batched['stimulus'])
            hidden_state = model.init_hidden(_batch_size)
            num_sample_points = sample_batched['stimulus'].shape[1]
         
            for i_time in range(num_sample_points):
                input_data = sample_batched['stimulus'][:, i_time].unsqueeze(dim=1).to(device)
                label_data = sample_batched['label'].float().to(device)
                hidden_state = hidden_state.to(device)
                output, hidden_state = model(input_data, hidden_state)   
             
            if i_time >=(num_sample_points - time_step_for_loss):
                test_loss += loss_fn(output.squeeze().to(device), label_data)

            correct += ((output.sign() * label_data) == 1).sum().item() / _batch_size

    test_loss /= num_epoches
    correct /= num_epoches
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
# train model 
loss_history = train(train_dataloader, model, loss_fn, optimizer)

In [ ]:
plt.plot(loss_history)
plt.xlabel('batch [-]')
plt.ylabel('error [-]')
plt.show()

In [ ]:
# test model 
test(test_dataloader, model, loss_fn)

# visualise dynamics of trained RNN
visualise_dynamics.plot_dynamics_each_trial(
    device, model, t_max, PDM_dataset_test, test_dataloader, num_trial_to_plot=2)

## 4. Analysis of selectivity and connectivity.

Compute covariance matrix in connectivity space. 

In [ ]:
input_vector = model.feedforward_input.weight.cpu().detach().numpy()
readout_vector = model.readout.weight.cpu().detach().numpy().T
input_selection_vector = model.recurrent_input.right_singular_vector.cpu().detach().numpy().T
output_vector = model.recurrent_input.left_singular_vector.cpu().detach().numpy()

# set connectivity data and compute cov matrix
connectivity_data = np.hstack((input_selection_vector, output_vector, input_vector, readout_vector))
connectivity_covariance_matrix = np.cov(connectivity_data.T)
vector_names = [rf'n$^{i}$' for i in range(1, model.rank+ 1)]
vector_names.extend([rf'm$^{i}$' for i in range(1, model.rank+ 1)])
vector_names.extend(['I', 'W'])

Plot neurons in connectivity space.

In [ ]:
visualise_connectivity.plot_neurons_connectivity_space(
    connectivity_data, connectivity_covariance_matrix, model.rank)

Visualise covariance matrix.

In [ ]:
clim = [-2.2, 2.2]
visualise_connectivity.plot_connectivity_covariance_matrix(
    connectivity_data, connectivity_covariance_matrix, model.rank, clim)

## 5. Gaussian mixture model 

Approximate neurons in the network by a single-population GMM.

In [ ]:
# fit to 4-d gaussian distribution with one population
GMM = GaussianMixture(n_components = 1, random_state=0)
GMM.fit(connectivity_data)

for i in range(connectivity_data.shape[1]):
    print(vector_names[i] + rf' mean: {GMM.means_[0][i]}')
    #GMM.covariances_

# resample from the fitted distribution
resampled_connectivity_data = \
    np.random.multivariate_normal(GMM.means_[0], GMM.covariances_[0], size=hidden_size)
print(f'shape of resampled_connectivity_data: {resampled_connectivity_data.shape}')

Visualise original neurons and re-sampled neurons.

In [ ]:
visualise_connectivity.plot_resampled_neurons_connectivity_space(
    resampled_connectivity_data, connectivity_data, connectivity_covariance_matrix, model.rank)

Test the network with the resampled connectivity.

In [ ]:
# Set up a network with the resampled connecitivity data.
resampled_model = \
    neural_networks.LowRankRNN(input_size, hidden_size, output_size, rank, time_step, tau).to(device)

# use resampled data for the model 
# input vector (hiddeni_size x input_size)
resampled_model.feedforward_input.weight = \
    torch.Tensor(resampled_connectivity_data[:, -2, np.newaxis])
# readout vector (output_size x hidden_size)
resampled_model.readout.weight = \
    torch.Tensor(resampled_connectivity_data[:, -1, np.newaxis].T)
# input selection vectors (rank x hidden_size)
resampled_model.recurrent_input.right_singular_vector = \
    torch.nn.Parameter(torch.Tensor(resampled_connectivity_data[:, 0:rank].T))
# output vectors (hidden_size x rank)
resampled_model.recurrent_input.left_singular_vector = \
    torch.nn.Parameter(torch.Tensor(resampled_connectivity_data[:, rank:-2]))

# test the resampled model
test(test_dataloader, resampled_model, loss_fn)
visualise_dynamics.plot_dynamics_each_trial(
    device, resampled_model, t_max, PDM_dataset_test, test_dataloader, num_trial_to_plot=2)

## 6. Project data onto the m-I subspace.

In [ ]:
sample_batched = next(iter(test_dataloader))
# run feedforward simulation and obtain the history of hidden state
hidden_state_history = np.empty((PDM_dataset_test.num_sample_points, batch_size, hidden_size))
for i_time in range(PDM_dataset_test.num_sample_points):
      if i_time == 0:
            hidden_state = model.init_hidden(batch_size)
      input_data = sample_batched['stimulus'][:, i_time].unsqueeze(dim=1)
      with torch.no_grad():
            output, hidden_state = model(input_data.to(device), hidden_state.to(device))
      hidden_state_history[i_time, :, :] = hidden_state.cpu().detach().numpy()

# project data to m 
projection_m = (hidden_state_history @ output_vector).squeeze()
# project data to I
projection_I = (hidden_state_history @ input_vector).squeeze()

In [ ]:
# plot trajectory of hidden state in the m-I plane
plt.scatter(projection_m[0, 0], projection_I[0, 0], s=50, color='k', label='start', zorder=10)
for i_trial in range(num_trial_to_plot):
    plt.plot(projection_m[:, i_trial], 
             projection_I[:, i_trial], 
             color=list_colors[i_trial],
             label=f'trials {i_trial}')
    plt.scatter(projection_m[-1, i_trial], 
                projection_I[-1, i_trial], 
                s=50, c=list_colors[i_trial], label='end', zorder=10)

plt.xlabel('latent variable k [-]')
plt.ylabel('latent variable u [-]')
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.show()

## 7. Dynamics with latent variables

In [ ]:
latent_model = latent_dynamics.LatentDynamics(rank, connectivity_covariance_matrix, time_step)

# run simulation using the first batch from test_dataloader 
i_batch = 0
sample_batched = next(iter(test_dataloader))
stimulus_batched = sample_batched['stimulus'].numpy()
label_batched = sample_batched['label'].numpy()
latent_model.run_simulation(stimulus_batched, label_batched)
correct = np.sum(label_batched == np.sign(latent_model.ks_history[:,0,-1])) / len(label_batched)
print(f"Latent dynamics: \n Accuracy: {(100*correct):>0.1f}%\n")

In [ ]:
visualise_dynamics.plot_latent_dynamics_each_trial_PDM(
    PDM_dataset_test.time, latent_model.ks_history, 
    stimulus_batched, label_batched, 
    i_batch, num_trial_to_plot=5)

## 8. Analyse locally linear dynamics around fixed points of a trained RNN
Find fixed points by minimising enery 